In [5]:
import yfinance as yf
import requests
import io
import pandas as pd

In [6]:
NASDAQLISTED_URL = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt"
OTHERLISTED_URL  = "https://www.nasdaqtrader.com/dynamic/symdir/otherlisted.txt"


In [ ]:
class TickerInfo:
    def __init__(self, urls: list[str], params = {"BATCH_SIZE": 500}):
        self.urls = urls
        self.params = params
        self.symbols = self.get_symbols()
        
    def get_symbols(self) -> list[str]:
        ls = []
        for url in self.urls:
            df = self._download_ticker_list(url)
            # also try 'NASDAQ Symbol' for otherlisted
            if 'NASDAQ Symbol' in df.columns:
                ls.extend(df['NASDAQ Symbol'].tolist())
            else:
                ls.extend(df['Symbol'].tolist())
        # Check for duplicates
        if len(ls) != len(set(ls)):
            print("Warning: Duplicate symbols found!")
        return ls
    
    def _download_ticker_list(self, url: str) -> pd.DataFrame:
        r = requests.get(url)
        r.raise_for_status()
        lines = [ln for ln in r.text.splitlines() if not ln.startswith('File')]
        data = io.StringIO("\n".join(lines))
        df = pd.read_csv(data, sep="|")
        df.apply(lambda x : x.replace('.','-'))
        return df
    
    def get_market_caps(self) -> dict[str, int]:
        market_caps = {}
        BATCH_SIZE = self.params.get("BATCH_SIZE", 500)
        for i in range(0, len(self.symbols), BATCH_SIZE):
            try:
                batch_symbols = self.symbols[i:i + BATCH_SIZE]
                batch_market_caps = self._fetch_market_caps(batch_symbols)
                market_caps.update(batch_market_caps)
            except Exception as e:
                print(f"Error fetching batch starting at index {i}: {e}")
        return market_caps


tickers = TickerInfo([NASDAQLISTED_URL, OTHERLISTED_URL])


In [26]:
tickers.get_market_caps()

Error fetching batch starting at index 0: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 500: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 1000: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 1500: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 2000: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 2500: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 3000: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 3500: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 4000: 'TickerInfo' object has no attribute '_fetch_market_caps'
Error fetching batch starting at index 4500: 'TickerInfo' object has no attri

{}

In [22]:
len(set(tickers))

12167

In [ ]:

ticker = yf.Ticker("AAPL")
market_cap = ticker.info["marketCap"]